Install the `event_aug` package according to the instructions in the README.

In [5]:
from event_aug.spike_encoding import delta_intensity_code_video
from event_aug.spike_injection import inject_event_spikes
from event_aug.utils import array_to_video, download_from_youtube, resize_video

In [6]:
import numpy as np
import os
import os
import json
import tables
import cv2
import stl

In [7]:
WORKING_DIR = "tmp"
os.makedirs(WORKING_DIR, exist_ok=True)

### Download video from YouTube

In [8]:
download_from_youtube(
    ["https://www.youtube.com/watch?v=cbL9NV-dStU"],
    start_times=[20],
    end_times=[110],
    save_dir=WORKING_DIR,
)

ffmpeg version 4.2.2 Copyright (c) 2000-2019 the FFmpeg developers
  built with gcc 7.3.0 (crosstool-NG 1.23.0.449-a04d0)
  configuration: --prefix=/home/neelay/miniconda3 --cc=/tmp/build/80754af9/ffmpeg_1587154242452/_build_env/bin/x86_64-conda_cos6-linux-gnu-cc --disable-doc --enable-avresample --enable-gmp --enable-hardcoded-tables --enable-libfreetype --enable-libvpx --enable-pthreads --enable-libopus --enable-postproc --enable-pic --enable-pthreads --enable-shared --enable-static --enable-version3 --enable-zlib --enable-libmp3lame --disable-nonfree --enable-gpl --enable-gnutls --disable-openssl --enable-libopenh264 --enable-libx264
  libavutil      56. 31.100 / 56. 31.100
  libavcodec     58. 54.100 / 58. 54.100
  libavformat    58. 29.100 / 58. 29.100
  libavdevice    58.  8.100 / 58.  8.100
  libavfilter     7. 57.100 /  7. 57.100
  libavresample   4.  0.  0 /  4.  0.  0
  libswscale      5.  5.100 /  5.  5.100
  libswresample   3.  5.100 /  3.  5.100
  libpostproc    55.  5.100

### Resize video to camera resolution

In [9]:
# Optional

resize_video(
    "tmp/0.mp4",
    "tmp/0_resized.mp4",
    size=(260, 346),
)

Original frame width: 1920, Original frame height: 1080
Resizing to 260x346


### Convert video to spikes

In [10]:
spikes = delta_intensity_code_video(
    "tmp/0_resized.mp4",
    percent_threshold=25,
    mode="percent_threshold",
    exclude_start=True,
    return_arr=True,
    save_video=True,
    video_save_path="tmp/encoded_vid_0.mp4",
    save_arr=True,
    arr_save_path="tmp/encoded_vid_0.npy",
)

Frame width: 260, Frame height: 346, Input FPS: 25, Output FPS: 25, Number of frames: 2251


In [11]:
spikes.shape, spikes.dtype

((2251, 346, 260), dtype('bool'))

In [12]:
np.unique(spikes, return_counts=True)

(array([False,  True]), array([177172928,  25327032]))

### Inject spikes into existing event sequence

In [13]:
event_file_path = "../projection/data/20220112_jpt_floating_kth_screwdriver/0002/orig_event.h5"
save_path = "../projection/data/20220112_jpt_floating_kth_screwdriver/0002/event.h5"

In [14]:
inject_event_spikes(
    event_file_path,
    save_path,
    spikes_arr=spikes, # "tmp/encoded_vid_0.npy",
    spikes_arr_from_file=False, # True
    memory_map=True, # False
    fps=100,
    label=0,
    polarity=1,
    resize_size=(260, 346),
    verbose=True,
)


Processing frame 0 of the event spikes video/array
Injecting event spikes found at 0 locations in the frame

Processing frame 1 of the event spikes video/array
Injecting event spikes found at 10961 locations in the frame

Processing frame 2 of the event spikes video/array
Injecting event spikes found at 10873 locations in the frame

Processing frame 3 of the event spikes video/array
Injecting event spikes found at 11146 locations in the frame

Processing frame 4 of the event spikes video/array
Injecting event spikes found at 11050 locations in the frame

Processing frame 5 of the event spikes video/array
Injecting event spikes found at 10762 locations in the frame

Processing frame 6 of the event spikes video/array
Injecting event spikes found at 10718 locations in the frame

Processing frame 7 of the event spikes video/array
Injecting event spikes found at 11079 locations in the frame

Processing frame 8 of the event spikes video/array
Injecting event spikes found at 10701 locations 

### Project object spikes and save to video

In [15]:
# OpenCV colours

white = (255, 255, 255)
blue = (255, 0, 0)
green = (0, 255, 0)
red = (0, 0, 255)
yellow = (0, 255, 255)
grey = (50, 50, 50)

In [16]:
def get_next_event(events_iter, camera):
    event = {}
    event[f'timestamp_{camera}'] = next(events_iter[f'timestamp_{camera}'])
    event[f'polarity_{camera}'] = next(events_iter[f'polarity_{camera}'])
    event[f'xy_undistorted_{camera}'] = next(events_iter[f'xy_undistorted_{camera}'])
    event[f'label_{camera}'] = next(events_iter[f'label_{camera}'])

    return event


def get_next_pose(poses_iter, n_cameras):
    pose = {}

    pose['timestamp'] = next(
        poses_iter['timestamp'])

    pose['rotation'] = {}
    for prop_name in poses_iter['rotation'].keys():
        pose['rotation'][prop_name] = next(
            poses_iter['rotation'][prop_name])

    for i in range(n_cameras):
        pose[f'camera_{i}_rotation'] = {}
        for prop_name in poses_iter[f'camera_{i}_rotation'].keys():
            pose[f'camera_{i}_rotation'][prop_name] = next(
                poses_iter[f'camera_{i}_rotation'][prop_name])

    pose['translation'] = {}
    for prop_name in poses_iter['translation'].keys():
        pose['translation'][prop_name] = next(
            poses_iter['translation'][prop_name])

    for i in range(n_cameras):
        pose[f'camera_{i}_translation'] = {}
        for prop_name in poses_iter[f'camera_{i}_translation'].keys():
            pose[f'camera_{i}_translation'][prop_name] = next(
                poses_iter[f'camera_{i}_translation'][prop_name])

    return pose

In [27]:
def projection(data_path, output_video_path, max_frames=500, fps=25):

    n_cameras = 1
    # n_cameras = 2
    event_distinguish_polarity = False

    dvs_cam_height = [np.uint32(260) for i in range(n_cameras)]
    dvs_cam_width = [np.uint32(346) for i in range(n_cameras)]
    dvs_cam_origin_x_offset = [dvs_cam_width[i] / 2 for i in range(n_cameras)]
    dvs_cam_origin_y_offset = [dvs_cam_height[i] / 2 for i in range(n_cameras)]
    dvs_cam_nominal_f_len = [4.0 for i in range(n_cameras)]
    dvs_cam_pixel_mm = [1.8e-2 for i in range(n_cameras)]

    # Read recording info from JSON
    with open(f'{data_path}/info.json', 'r') as info_json_file:
        info_json = json.load(info_json_file)

    ##################################################################

    # === READ PROPS DATA ===
    props_markers = {}    # contains the translation of each marker, relative to prop origin
    props_meshes = {}     # contains prop STL meshes (polygon, translation, vertex)
    props_labels = {}     # contains integer > 0 class labels of the props
    props_dilation = {}   # contains dilation kernels for the mask of each prop

    props_names = list(info_json['prop_marker_files'].keys())
    for prop_name in props_names:
        with open(info_json['prop_marker_files'][prop_name], 'r') as marker_file:
            markers = json.load(marker_file)
        props_markers[prop_name] = markers
        mesh = stl.mesh.Mesh.from_file(info_json['prop_mesh_files'][prop_name]).vectors.transpose(0, 2, 1)
        props_meshes[prop_name] = mesh
        props_labels[prop_name] = info_json['prop_labels'][prop_name]
        props_dilation[prop_name] = np.ones((3, 3), 'uint8')

    # Change prop mask dilation
    #props_dilation['kth_hammer'] = np.ones((4, 4), 'uint8')
    #props_dilation['kth_screwdriver'] = np.ones((4, 4), 'uint8')
    #props_dilation['kth_spanner'] = np.ones((4, 4), 'uint8')

    ##################################################################

    # === READ CALIBRATION FILES ===

    path_projection = info_json['projection_calibration_path']

    v_to_dvs_rotation_file = [f'{path_projection}/v_to_dv_{i}_rotation.npy' for i in range(n_cameras)]
    v_to_dvs_rotation = [np.load(name) for name in v_to_dvs_rotation_file]

    v_to_dvs_translation_file = [f'{path_projection}/v_to_dv_{i}_translation.npy' for i in range(n_cameras)]
    v_to_dvs_translation = [np.load(name) for name in v_to_dvs_translation_file]

    v_to_dvs_f_len_scale_file = [f'{path_projection}/v_to_dv_{i}_focal_length_scale.npy' for i in range(n_cameras)]
    v_to_dvs_f_len_scale = [np.load(name) for name in v_to_dvs_f_len_scale_file]
    v_to_dvs_f_len = [dvs_cam_nominal_f_len[i] * v_to_dvs_f_len_scale[i] for i in range(n_cameras)]

    v_to_dvs_x_scale_file = [f'{path_projection}/v_to_dv_{i}_x_scale.npy' for i in range(n_cameras)]
    v_to_dvs_x_scale = [np.load(name) for name in v_to_dvs_x_scale_file]

    ##################################################################



    # initialise temp memory
    event_pos = [np.zeros((dvs_cam_height[i], dvs_cam_width[i]), dtype='uint64') for i in range(n_cameras)]
    event_neg = [np.zeros((dvs_cam_height[i], dvs_cam_width[i]), dtype='uint64') for i in range(n_cameras)]
    event_image = [np.zeros((dvs_cam_height[i], dvs_cam_width[i], 3), dtype='uint8') for i in range(n_cameras)]
    prop_masks = [{prop_name: np.empty((dvs_cam_height[i], dvs_cam_width[i]), dtype='uint8')
                   for prop_name in props_names} for i in range(n_cameras)]

    events_vid = np.zeros((1, dvs_cam_height[0], dvs_cam_width[0], 3), dtype='uint8')

    # load DVS event data
    events_file_name = f'{data_path}/event.h5'
    events_file = tables.open_file(events_file_name, mode='r')
    events_iter = []
    for i in range(n_cameras):
        e_iter = {}
        e_iter[f'timestamp_{i}'] = events_file.root[f'timestamp_{i}'].iterrows()
        e_iter[f'polarity_{i}'] = events_file.root[f'polarity_{i}'].iterrows()
        e_iter[f'xy_undistorted_{i}'] = events_file.root[f'xy_undistorted_{i}'].iterrows()
        e_iter[f'label_{i}'] = events_file.root[f'label_{i}'].iterrows()
        events_iter.append(e_iter)

    event = [get_next_event(events_iter[i], i) for i in range(n_cameras)]

    # load Vicon pose data file
    poses_file_name = f'{data_path}/pose.h5'
    poses_file = tables.open_file(poses_file_name, mode='r')
    poses_iter = {}

    timestamp = poses_file.root.timestamp
    poses_iter['timestamp'] = timestamp.iterrows()

    poses_iter['rotation'] = {}
    for i in range(n_cameras):
        poses_iter[f'camera_{i}_rotation'] = {}

    poses_iter['translation'] = {}
    for i in range(n_cameras):
        poses_iter[f'camera_{i}_translation'] = {}

    for prop_name in props_names:
        rotation = poses_file.root.props[prop_name].rotation
        poses_iter['rotation'][prop_name] = rotation.iterrows()
        for i in range(n_cameras):
            cam_rotation = poses_file.root.props[prop_name][f'camera_{i}_rotation']
            poses_iter[f'camera_{i}_rotation'][prop_name] = cam_rotation.iterrows()

        translation = poses_file.root.props[prop_name].translation
        poses_iter['translation'][prop_name] = translation.iterrows()
        for i in range(n_cameras):
            cam_translation = poses_file.root.props[prop_name][f'camera_{i}_translation']
            poses_iter[f'camera_{i}_translation'][prop_name] = cam_translation.iterrows()

    pose = get_next_pose(poses_iter, n_cameras)

    # === MAIN LOOP ===
    
    frames_count = 0

    done_event = [False for i in range(n_cameras)]
    while not all(done_event):

        try:
            pose_new = get_next_pose(poses_iter, n_cameras)
            pose_midway = pose['timestamp'] / 2 + pose_new['timestamp'] / 2
        except StopIteration:
            print('DEBUG: out of Vicon poses')
            break

        frames_count += 1
        if frames_count > max_frames:
            break

        # print()
        # print('Vicon pose timestamp: ', pose['timestamp'])
        print(f"Processing frame {frames_count}")

        for prop_name in props_names:

            # compute prop mask for each camera
            for i in range(n_cameras):
                prop_masks[i][prop_name].fill(0)

                mesh_to_dvs_rotation = pose[f'camera_{i}_rotation'][prop_name]
                mesh_to_dvs_translation = pose[f'camera_{i}_translation'][prop_name]

                if not np.isfinite(mesh_to_dvs_rotation).all() or not np.isfinite(mesh_to_dvs_translation).all():
                    continue

                # transform to DVS camera space
                dvs_space_p = np.matmul(mesh_to_dvs_rotation, props_meshes[prop_name]) + mesh_to_dvs_translation
                dvs_space_p[:, :2, :] *= (1 / dvs_space_p[:, np.newaxis, 2, :])
                dvs_space_p = dvs_space_p[:, :2, :]
                dvs_space_p *= v_to_dvs_f_len[i]
                dvs_space_p /= dvs_cam_pixel_mm[i]
                dvs_space_p *= v_to_dvs_x_scale[i]
                dvs_space_p += [[dvs_cam_origin_x_offset[i]], [dvs_cam_origin_y_offset[i]]]
                dvs_space_p_int = np.rint(dvs_space_p).astype('int32')

                # transpose points for OpenCV
                dvs_space_p_int = dvs_space_p_int.transpose(0, 2, 1)

                # compute prop mask
                cv2.fillPoly(prop_masks[i][prop_name], dvs_space_p_int, 255)
                prop_masks[i][prop_name] = cv2.dilate(prop_masks[i][prop_name], props_dilation[prop_name])


        # process DVS events
        for i in range(n_cameras):
            if not done_event[i]:

                image = event_image[i]
                pos = event_pos[i]
                neg = event_neg[i]

                image.fill(0)
                pos.fill(0)
                neg.fill(0)

                while event[i][f'timestamp_{i}'] < pose_midway:
                    xy_int = np.rint(event[i][f'xy_undistorted_{i}']).astype('int32')

                    if event[i][f'polarity_{i}']:
                        pos[xy_int[1], xy_int[0]] += 1
                    else:
                        neg[xy_int[1], xy_int[0]] += 1

                    if event_distinguish_polarity:
                        if event[i][f'polarity_{i}']:
                            image[xy_int[1], xy_int[0]] = white # red
                        else:
                            image[xy_int[1], xy_int[0]] = white # green
                    else:
                        image[xy_int[1], xy_int[0]] = white # green


                    # get event label
                    label = event[i][f'label_{i}']

                    try:
                        event[i] = get_next_event(events_iter[i], i)
                    except StopIteration:
                        print(f'DEBUG: out of DVS {i} events')
                        done_event[i] = True
                        break

                # fill DVS event image with events, then mask it
                for prop_name in props_names:
                    # TODO: FIXME: image[mask] = grey will overwrite events from other props
                    mask = prop_masks[i][prop_name].astype('bool')
                    image[mask] = grey # show prop mask?
                    if event_distinguish_polarity:
                        mask_neg = neg > pos
                        image[(mask_neg & mask)] = blue
                        mask_pos = pos > neg
                        image[(mask_pos & mask)] = yellow
                    else:
                        mask_pos_neg = neg.astype('bool') | pos.astype('bool')
                        image[(mask_pos_neg & mask)] = red

                img = image.copy()
                img = np.expand_dims(img, axis=0)        
                events_vid = np.vstack((events_vid, img))


        pose = pose_new


    array_to_video(events_vid, output_video_path, fps)

    # cleanup
    events_file.close()
    poses_file.close()

In [18]:
# Switch to directory containing data

%cd ../projection
!ls

/home/neelay/projects/gsoc/projection
camera_calibration    gen_aug_images.py       props
custom_projection.py  inject.py		      tinker.ipynb
data		      projection_calibration  tmp


In [19]:
data_path = "data/20220112_jpt_floating_kth_screwdriver/0002/"

# Should contain:
# - event.h5 (with event injected from the previous step)
# - pose.h5
# - info.json

# The other data files containing translation data, rotation data, etc. should be present
# as per the info (paths) in the info.json file.

In [28]:
projection(
    data_path=data_path,
    output_video_path="../event_aug/tmp/augmented_event.mp4",
    max_frames=600,
    fps=30,
)

Processing frame 1
Processing frame 2
Processing frame 3
Processing frame 4
Processing frame 5
Processing frame 6
Processing frame 7
Processing frame 8
Processing frame 9
Processing frame 10
Processing frame 11
Processing frame 12
Processing frame 13
Processing frame 14
Processing frame 15
Processing frame 16
Processing frame 17
Processing frame 18
Processing frame 19
Processing frame 20
Processing frame 21
Processing frame 22
Processing frame 23
Processing frame 24
Processing frame 25
Processing frame 26
Processing frame 27
Processing frame 28
Processing frame 29
Processing frame 30
Processing frame 31
Processing frame 32
Processing frame 33
Processing frame 34
Processing frame 35
Processing frame 36
Processing frame 37
Processing frame 38
Processing frame 39
Processing frame 40
Processing frame 41
Processing frame 42
Processing frame 43
Processing frame 44
Processing frame 45
Processing frame 46
Processing frame 47
Processing frame 48
Processing frame 49
Processing frame 50
Processin